In [1]:
# -*- coding: utf-8 -*-
"""
@Author: Guan-Fu Liu
Created on Jan. 23, 2024
Last modified on Feb. 3, 2024

Convert the stellar yield file from Karakas et al. (2010) to the format more convenient for 
further IMF determination.

Karakas et al. (2010) provides both net yields and total yields, which will be extracted separately.

The yield files can be downloaded from https://academic.oup.com/mnras/article/403/3/1413/1048800
"""
import numpy as np
import pandas as pd
import pyatomdb
import re
import os
from functools import reduce

In [2]:
def merge_two_dfs(left, right):
    """
    Define a function to merge two DataFrames
    """
    return pd.merge(left, right, how='outer')

In [3]:
yield_dir = './Original/K10/'
out_dir = './AGB/'
yield_files = ['table_a2.txt', 'table_a3.txt', 'table_a4.txt',
               'table_a5.txt']  # Yields files of different initial metallicity.
# 'table_a6.txt' is for the partial mixing zone model
for file in yield_files:
    with open(yield_dir+file) as f:
        lines = f.readlines()
    Z_ini = re.search(r"Z=(\d+(\.\d+)?),", lines[0].replace(" ", "")).group(1)  # The initial metallicity
    row_sep = [ ]  # separate the yields of stars with different initial mass
    M_ini = [ ]  # initial mass of the stars
    for i, line in enumerate(lines):
        if line.startswith('#'):
            row_sep.append(i)
            M_ini.append(re.search(r"Minitial=(\d+(\.\d+)?)msun,", line.replace(" ", "")).group(1))
        else:
            pass
    
    print('The current yield file is %s, the initial metallicity is %s, and the number of' % (file, Z_ini) + 
          'initial masses are %d.' % (len(M_ini)))
    tables = { }
    for i in range(0, len(row_sep), 1):
        # Read the table
        if i < len(row_sep)-1:
            table1 = pd.read_table(yield_dir+file, sep=r'\s+', skiprows=row_sep[i]+1, 
                            nrows=row_sep[i+1]-row_sep[i]-2, header=0)
        else:
            table1 = pd.read_table(yield_dir+file, sep=r'\s+', skiprows=row_sep[i]+1, header=0)
        # # Drop the row of 'g' and 'n' species ('g' is gluon, 'n' is neutron)
        table1 = table1.loc[~table1.loc[:,'species'].isin(['g', 'n']), ['species', 'A', 'yield', 'mass(i)_lost']]
        table1.loc[:, 'species'].replace('p', 'h1', inplace=True)  # replace 'p' with 'h1'
        table1.loc[:, 'species'].replace('d', 'h2', inplace=True)  # replace 'd' with 'h2'
        # Convert the species like 'h1' to element 'H'
        table1.loc[:, 'species'] = table1.loc[:,'species'].str.capitalize()
        table1.loc[:, 'species'] = table1.loc[:, 'species'].str.extract(r'([A-Za-z]+)').to_numpy().ravel()
        # Rename the 'species' column to 'element'
        table1.rename(columns={'species':'element'}, inplace=True)
        # Convert the yields in mass to in number
        table1.loc[:, ['yield', 'mass(i)_lost']] = table1.loc[:, ['yield', 'mass(i)_lost']].div(table1.loc[:, 'A'], axis=0)
        # The 'yield' column is the net yield,
        # while the 'mass(i)_lost' column is the total yield.
        # Rename the 'yield' to 'net' (net yield)
        # and the 'mass(i)_lost' to 'total' (total yield).
        table1.rename(columns={'yield':'net', 'mass(i)_lost':'total'}, inplace=True)
        # Drop the 'A' column
        table1.drop(columns='A', inplace=True)
        # Sum up the number of net and total yields for each element
        table1 = table1.groupby(by = 'element').sum()
        # Reset index
        table1.reset_index(inplace=True)
        # Add the atomic number column
        table1['Z'] = table1['element'].apply(pyatomdb.atomic.elsymb_to_Z)
        # Sort rows by the atomic number
        table1.sort_values(by='Z', inplace=True)
        # Drop the atomic number column
        table1.drop(columns='Z', inplace=True)
        # Reset index
        table1.reset_index(drop=True, inplace=True)
        tables[M_ini[i]] = table1


    tables_net = [ ]  # Tables of net yield, change the column name 'net' to initial mass
    for m in M_ini:
        tmp = tables[m].loc[:, ['element', 'net']]
        tmp.rename(columns={'net':m}, inplace=True)
        tables_net.append(tmp)

    # Merge tables of net yields of different inital masses to one table
    merged_net = reduce(merge_two_dfs, tables_net)
    merged_net.set_index('element', inplace=True)
    # Sort the columns (different initial masses)
    sorted_cols = merged_net.columns[merged_net.columns.to_numpy().astype(float).argsort()]
    merged_net = merged_net.loc[:, sorted_cols.to_list()]
    merged_net.to_csv(out_dir+'K10-'+Z_ini+'-AGB-net.csv', float_format='%.4e')


    tables_tot = [ ]  # Tables of total yield, change the column name 'total' to initial mass
    for m in M_ini:
        tmp = tables[m].loc[:, ['element', 'total']]
        tmp.rename(columns={'total':m}, inplace=True)
        tables_tot.append(tmp)

    # Merge tables of total yields of different inital masses to one table
    merged_tot = reduce(merge_two_dfs, tables_tot)
    merged_tot.set_index('element', inplace=True)
    # Sort the columns (different initial masses)
    sorted_cols = merged_tot.columns[merged_tot.columns.to_numpy().astype(float).argsort()]
    merged_tot = merged_tot.loc[:, sorted_cols.to_list()]
    merged_tot.to_csv(out_dir+'K10-'+Z_ini+'-AGB-total.csv', float_format='%.4e')

The current yield file is table_a2.txt, the initial metallicity is 0.0200, and the number ofinitial masses are 16.
The current yield file is table_a3.txt, the initial metallicity is 0.0080, and the number ofinitial masses are 15.
The current yield file is table_a4.txt, the initial metallicity is 0.0040, and the number ofinitial masses are 15.
The current yield file is table_a5.txt, the initial metallicity is 0.0001, and the number ofinitial masses are 16.
